In [ ]:
import matplotlib.dates as mdates
import matplotlib.cbook as cbook
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
import subprocess  # To run shell commands
import csv
import datetime
import subprocess
import os
import pandas as pd
import numpy as np
import holidays
import glob
from pathlib import Path

# Install necessary Python libraries locally (instead of using Colab's `!pip`)
subprocess.run(["pip", "install", "geopy"], check=True)
subprocess.run(["pip", "install", "pyjanitor"], check=True)
subprocess.run(["pip", "install", "scikit-learn"], check=True)
subprocess.run(["pip", "install", "shapely"], check=True)
subprocess.run(["pip", "install", "holidays"], check=True)


In [ ]:
# inflation data - Bank of Canada

# Clean inflation data

# Define the local repository path
repo_dir = r"C:\Users\AShresth\Data-Challenge"

# Change the working directory to your local repository
os.chdir(repo_dir)

# Run the git pull command to update the local repository with the latest
# changes
subprocess.run(["git", "pull", "origin", "main"], check=True)

# Define the path to the inflation data CSV file within the repository
path = os.path.join(repo_dir, "adjusted_inflation.csv")

# Load the inflation data CSV
df_inflca = pd.read_csv(path)


df_inflca.rename(
    columns={
        "REF_DATE": "date",
        "VALUE": "adjusted national inflation rate",
        "Alternative measures": "measure",
    },
    inplace=True,
)

df_inflca["date"] = pd.to_datetime(df_inflca["date"])
df_inflca = df_inflca[df_inflca["date"].dt.year > 2014]
df_inflca = df_inflca[df_inflca.UOM_ID == 435]
df_inflca = df_inflca[
    df_inflca.measure
    == "Measure of core inflation based on a weighted median approach, CPI-median (index, 198901=100)"
]

df_inflca.rename(
    columns={
        "REF_DATE": "date",
        "VALUE": "adjusted national inflation rate",
    },
    inplace=True,
)

df_inflca = df_inflca[["date", "adjusted national inflation rate"]]
df_inflca


In [ ]:
# inflation data - statcan

# Clean uninflation data

# Define the local repository path
repo_dir = r"C:\Users\AShresth\Data-Challenge"

# Change the working directory to your local repository
os.chdir(repo_dir)

# Run the git pull command to update the local repository with the latest
# changes
subprocess.run(["git", "pull", "origin", "main"], check=True)

# Define the path to the uninflation data CSV file within the repository
path = os.path.join(repo_dir, "unadjusted_inflation.csv")

# Load the inflation data CSV
df_infl = pd.read_csv(path)

df_infl["REF_DATE"] = pd.to_datetime(df_infl["REF_DATE"])
df_infl = df_infl[df_infl["REF_DATE"].dt.year > 2014]
df_infl = df_infl[df_infl.UOM_ID == 17]
df_infl = df_infl[df_infl["Products and product groups"].isin(["All-items", "Shelter"])]

df_infl.rename(
    columns={
        "GEO": "CMA",
        "REF_DATE": "date",
        "Products and product groups": "product",
        "VALUE": "inflation rate",
    },
    inplace=True,
)

df_infl["CMA"].replace(
    {
        "Canada": "CA",
        "Newfoundland and Labrador": "NL",
        "St. John's, Newfoundland and Labrador": "St. John's",
        "Prince Edward Island": "PE",
        "Charlottetown and Summerside, Prince Edward Island": "Charlottetown",
        "Nova Scotia": "NS",
        "Halifax, Nova Scotia": "Halifax",
        "New Brunswick": "NB",
        "Saint John, New Brunswick": "Saint John",
        "Quebec": "QC",
        "Québec, Quebec": "Quebec",
        "Montréal, Quebec": "Montreal",
        "Ontario": "ON",
        "Ottawa-Gatineau, Ontario part, Ontario/Quebec": "Ottawa",
        "Toronto, Ontario": "Toronto",
        "Thunder Bay, Ontario": "Thunder Bay",
        "Manitoba": "MB",
        "Winnipeg, Manitoba": "Winnipeg",
        "Saskatchewan": "SK",
        "Regina, Saskatchewan": "Regina",
        "Saskatoon, Saskatchewan": "Saskatoon",
        "Alberta": "AB",
        "Edmonton, Alberta": "Edmonton",
        "Calgary, Alberta": "Calgary",
        "British Columbia": "BC",
        "Vancouver, British Columbia": "Vancouver",
        "Victoria, British Columbia": "Victoria",
        "Whitehorse, Yukon": "Whitehorse",
        "Yellowknife, Northwest Territories": "Yellowknife",
        "Iqaluit, Nunavut": "Iqualuit",
    },
    inplace=True,
)

df_infl = df_infl[["date", "CMA", "product", "inflation rate"]]
df_infl = df_infl.pivot_table("inflation rate", ["date", "CMA"], "product")
df_infl = df_infl.reset_index()

df_infl.rename(
    columns={
        "All-items": "inflation_all",
        "Shelter": "inflation_shelter",
    },
    inplace=True,
)

df_infl


In [ ]:
# employment data

# Clean inflation data

# Define the local repository path
repo_dir = r"C:\Users\AShresth\Data-Challenge"

# Change the working directory to your local repository
os.chdir(repo_dir)

# Run the git pull command to update the local repository with the latest
# changes
subprocess.run(["git", "pull", "origin", "main"], check=True)

# Define the path to the uninflation data CSV file within the repository
path = os.path.join(repo_dir, "adjusted_employment.csv")

# Load the inflation data CSV
df_empl = pd.read_csv(path)


df_empl["REF_DATE"] = pd.to_datetime(df_empl["REF_DATE"])
df_empl = df_empl[df_empl["REF_DATE"].dt.year > 2014]
df_empl = df_empl[df_empl["Labour force characteristics"].isin(["Unemployment rate"])]
df_empl = df_empl[df_empl["Statistics"].isin(["Estimate"])]
df_empl = df_empl[df_empl["Data type"].isin(["Seasonally adjusted"])]


df_empl.rename(
    columns={"GEO": "province", "REF_DATE": "date", "VALUE": "unemployment rate"},
    inplace=True,
)

df_empl["province"].replace(
    {
        "Canada": "CA",
        "Newfoundland and Labrador": "NL",
        "Prince Edward Island": "PE",
        "Nova Scotia": "NS",
        "New Brunswick": "NB",
        "Quebec": "QC",
        "Ontario": "ON",
        "Manitoba": "MB",
        "Saskatchewan": "SK",
        "Alberta": "AB",
        "British Columbia": "BC",
    },
    inplace=True,
)

df_empl = df_empl[["date", "province", "unemployment rate"]]


In [ ]:
# employment data

# Clean inflation data

# Define the local repository path
repo_dir = r"C:\Users\AShresth\Data-Challenge"

# Change the working directory to your local repository
os.chdir(repo_dir)

# Run the git pull command to update the local repository with the latest
# changes
subprocess.run(["git", "pull", "origin", "main"], check=True)

# Define the path to the uninflation data CSV file within the repository
path = os.path.join(repo_dir, "unadjusted_employment.csv")

# Load the inflation data CSV
df_empl = pd.read_csv(path)

df_empl["REF_DATE"] = pd.to_datetime(df_empl["REF_DATE"])
df_empl = df_empl[df_empl["REF_DATE"].dt.year > 2014]
df_empl = df_empl[df_empl["Labour force characteristics"].isin(["Unemployment rate"])]
df_empl = df_empl[df_empl["Sex"].isin(["Both sexes"])]
df_empl = df_empl[df_empl["Age group"].isin(["15 years and over"])]
df_empl = df_empl[
    df_empl["Population centre and rural areas"].isin(
        ["Census metropolitan area and census agglomeration"]
    )
]

df_empl.rename(
    columns={"GEO": "province", "REF_DATE": "date", "VALUE": "unemployment rate"},
    inplace=True,
)

df_empl["province"].replace(
    {
        "Canada": "CA",
        "Newfoundland and Labrador": "NL",
        "Prince Edward Island": "PE",
        "Nova Scotia": "NS",
        "New Brunswick": "NB",
        "Quebec": "QC",
        "Ontario": "ON",
        "Manitoba": "MB",
        "Saskatchewan": "SK",
        "Alberta": "AB",
        "British Columbia": "BC",
    },
    inplace=True,
)

df_empl = df_empl[["date", "province", "unemployment rate"]]

df_empl


In [ ]:
# Clean Alberta Data


# Install openpyxl to read Excel files
subprocess.run(["pip", "install", "openpyxl"], check=True)

# Define the local repository path
repo_dir = r"C:\Users\AShresth\Data-Challenge"

# Change the working directory to your local repository
os.chdir(repo_dir)

# Run the git pull command to update the local repository with the latest
# changes
subprocess.run(["git", "pull", "origin", "main"], check=True)

# Define the path to the Alberta data Excel file within the repository
path = os.path.join(repo_dir, "shelter_alberta.xlsx")

# Load the Alberta shelter data from the Excel file
df_alberta = pd.read_excel(path, engine="openpyxl")

# Assign the dataframe to df
df = df_alberta

# Display the first few rows of the dataframe
df.head()


In [ ]:
# alberta
df_alberta = df

# find day of the week
df_alberta["date"] = pd.to_datetime(df["date"])
df_alberta["day_week"] = df_alberta["date"].dt.dayofweek
df_alberta["day_week"].replace(
    {
        0: "1-mon",
        1: "2-tue",
        2: "3-wed",
        3: "4-thu",
        4: "5-fri",
        5: "6-sat",
        6: "7-sun",
    },
    inplace=True,
)

# find month
df_alberta["month"].replace(
    {
        1: "01-jan",
        2: "02-feb",
        3: "03-mar",
        4: "04-apr",
        5: "05-may",
        6: "06-jun",
        7: "07-jul",
        8: "08-aug",
        9: "09-sep",
        10: "10-oct",
        11: "11-nov",
        12: "12-dec",
    },
    inplace=True,
)

# find last week of the month
df_alberta["Date_Month_End"] = df_alberta["date"] + pd.offsets.MonthEnd(n=0)
df_alberta["Date_Month_End"] = df_alberta["Date_Month_End"].dt.day
df_alberta["day_month"] = df_alberta["date"].dt.day
df_alberta["last_week"] = np.where(
    (df_alberta["Date_Month_End"] - df_alberta["day_month"]) < 8, 1, 0
)

# find first week of the month
df_alberta["first_week"] = np.where(df_alberta["day_month"] < 8, 1, 0)


# find provincial holidays
# df_alberta['hols'] = pd.Series(df.index).apply(lambda x: holidays.CountryHoliday('CAN',prov='ON').get(x)).values

# df_alberta['hols_bi'] = df_alberta['hols']
# df_alberta['hols_bi'] = np.where(df_alberta['hols'] == "", 0, 1)

# remove closed shelters
df_alberta.loc[df_alberta["Capacity"] == 0, "Capacity"] = np.nan
df_alberta["occupancy_rate"] = df_alberta["Overnight"] / df_alberta["Capacity"]

df_alberta = df_alberta[
    [
        "date",
        "city",
        "ShelterType",
        "ShelterName",
        "Organization",
        "Capacity",
        "Overnight",
        "year",
        "month",
        "occupancy_rate",
        "day_week",
        "last_week",
        "first_week",
    ]
]

df_alberta["date"] = df_alberta["date"].astype("datetime64[ns]")

df_alberta


In [ ]:
# merge weather files

# Install necessary dependencies
subprocess.run(["pip", "install", "openpyxl"], check=True)

# Define the local repository path
repo_dir = r"C:\Users\AShresth\Data-Challenge"

# Change the working directory to your local repository
os.chdir(repo_dir)

# Run the git pull command to update the local repository with the latest
# changes
subprocess.run(["git", "pull", "origin", "main"], check=True)

# Define the path to the weather data CSV files within the local repository
path = os.path.join(repo_dir, "weather", "*.csv")

# Get the list of all CSV files
all_files = glob.glob(path)

# Create an empty list to store dataframes
flist = []

# Loop through each file, read it into a dataframe, and append to the list
for filename in all_files:
    df_temp = pd.read_csv(filename, index_col=False)
    flist.append(df_temp)

# Concatenate all the individual dataframes into one dataframe
df_weather = pd.concat(flist, axis=0, ignore_index=True)

# Remove entries earlier than 2014
df_weather = df_weather[df_weather["LOCAL_YEAR"] >= 2014]

# Rename station names to city names
df_weather["STATION_NAME"].replace(
    {
        "BOW VALLEY": "Bow Valley",
        "CALGARY INTL A": "Calgary",
        "COLD LAKE A": "Cold Lake",
        "EDMONTON INTERNATIONAL CS": "Edmonton",
        "EDSON": "Edson",
        "FORT MCMURRAY A": "Fort McMurray",
        "GRANDE PRAIRIE A": "Grande Prairie",
        "LAC LA BICHE CLIMATE": "Lac La Biche",
        "LETHBRIDGE": "Lethbridge",
        "LLOYDMINSTER": "Lloydminster",
        "MEDICINE HAT": "Medicine Hat",
        "PEACE RIVER A": "Peace River",
        "RED DEER A": "Red Deer",
        "RED DEER REGIONAL A": "Red Deer",
        "SLAVE LAKE": "Slave Lake",
        "WETASKIWIN AGCM": "Wetaskiwin",
    },
    inplace=True,
)

# Remove time from date and keep only the date part
df_weather["LOCAL_DATE"] = pd.to_datetime(df_weather["LOCAL_DATE"]).dt.date

# Sort data by station name and date
df_weather = df_weather.sort_values(by=["STATION_NAME", "LOCAL_DATE"])

# Interpolate missing temperature data
df_weather["MIN_TEMPERATURE"] = df_weather["MIN_TEMPERATURE"].interpolate()
df_weather["MAX_TEMPERATURE"] = df_weather["MAX_TEMPERATURE"].interpolate()
df_weather["MEAN_TEMPERATURE"] = df_weather["MEAN_TEMPERATURE"].interpolate()

# Replace missing precipitation values with 0
df_weather.update(df_weather[["TOTAL_PRECIPITATION"]].fillna(0))

# Keep relevant columns
df_weather = df_weather[
    [
        "STATION_NAME",
        "MEAN_TEMPERATURE",
        "TOTAL_PRECIPITATION",
        "LOCAL_DATE",
        "MIN_TEMPERATURE",
        "MAX_TEMPERATURE",
        "PROVINCE_CODE",
        "x",
        "y",
    ]
]

# Rename columns to more user-friendly names
df_weather.rename(
    columns={
        "STATION_NAME": "city",
        "MEAN_TEMPERATURE": "mean_temperature",
        "TOTAL_PRECIPITATION": "total_precipitation",
        "LOCAL_DATE": "date",
        "MIN_TEMPERATURE": "min_temperature",
        "MAX_TEMPERATURE": "max_temperature",
        "PROVINCE_CODE": "province",
    },
    inplace=True,
)

# Ensure the 'date' column is in the correct datetime format
df_weather["date"] = df_weather["date"].astype("datetime64[ns]")

# Display the cleaned dataframe
df_weather.head()


In [ ]:
# Define the local repository path
repo_dir = r"C:\Users\AShresth\Data-Challenge"

# Change the working directory to your local repository
os.chdir(repo_dir)

# Run the git pull command to update the local repository with the latest
# changes
subprocess.run(["git", "pull", "origin", "main"], check=True)

# Define the path to the census data CSV file within the repository
path = os.path.join(repo_dir, "census.csv")

# Load the census data CSV
df_census = pd.read_csv(path, encoding="ISO-8859-1")

# keep only census
df_census["GEO_NAME"].replace(
    {
        "Calgary, City (CY)": "Calgary",
        "Cold Lake, City (CY)": "Cold Lake",
        "Drayton Valley, Town (T)": "Drayton Valley",
        "Edmonton, City (CY)": "Edmonton",
        "Edson, Town (T)": "Edson",
        "Grande Prairie, City (CY)": "Grande Prairie",
        "Lac la Biche County, Specialized municipality (SM)": "Lac La Biche",
        "Leduc, City (CY)": "Leduc",
        "Lethbridge, City (CY)": "Lethbridge",
        "Lloydminster (Part), City (CY)": "Lloydminster",
        "Medicine Hat, City (CY)": "Medicine Hat",
        "Peace River, Town (T)": "Peace River",
        "Red Deer, City (CY)": "Red Deer",
        "Slave Lake, Town (T)": "Slave Lake",
        "Spring Lake, Village (VL)": "Spruce Grove",
        "Wetaskiwin, City (CY)": "Wetaskiwin",
    },
    inplace=True,
)

list_cities = sorted(df_alberta["city"].unique().tolist())

df_census = df_census.loc[df_census["GEO_NAME"].isin(list_cities)]


list_characteristics = [
    1,
    3,
    4,
    5,
    35,
    36,
    37,
    40,
    41,
    42,
    43,
    44,
    45,
    46,
    47,
    48,
    49,
    50,
    51,
    52,
    53,
    54,
    55,
    57,
    252,
    253,
    360,
    361,
    363,
    364,
    379,
    380,
    381,
    382,
    1482,
    1483,
    1484,
    1486,
    1488,
    1490,
    1491,
    1492,
    1494,
    1496,
    1497,
    1498,
    1499,
    1527,
    1528,
    1529,
    1530,
    1531,
    1532,
    1533,
    1535,
    1536,
    1537,
    1983,
    1984,
    1985,
    1986,
    1987,
    1988,
    1991,
    2014,
    2015,
    2016,
    2017,
    2228,
    2229,
    2230,
    2239,
    2240,
    2241,
    2242,
    2245,
]
df_census = df_census.loc[df_census["CHARACTERISTIC_ID"].isin(list_characteristics)]

# keep relevant columns
df_census = df_census[
    [
        "CENSUS_YEAR",
        "GEO_NAME",
        "CHARACTERISTIC_ID",
        "CHARACTERISTIC_NAME",
        "C1_COUNT_TOTAL",
    ]
]

df_census.rename(columns={"CENSUS_YEAR": "year", "GEO_NAME": "city"}, inplace=True)

# create pivot table
df_census = pd.pivot_table(
    df_census, values="C1_COUNT_TOTAL", index=["city"], columns=["CHARACTERISTIC_NAME"]
)
df_census = df_census.reset_index()

# clean pivot table
df_census.rename(
    columns={
        "      2011 to 2015": "immigrant status - immigrants - 2011-2015",
        "      2016 to 2021": "immigrant status - immigrants - 2016-202l",
        "      External migrants": "migrant 5 years - movers - migrants - external",
        "      Internal migrants": "migrant 5 years - movers - migrants - internal",
        "      Permanent position": "workers - empoyee - permanent",
        "      Temporary position": "workers - employee - temporary",
        "    1980 to 1990": "immigrant status - immigrants - 1980-1990",
        "    1991 to 2000": "immigrant status - immigrants - 1991-2000",
        "    2001 to 2010": "immigrant status - immigrants - 2001-2010",
        "    Before 1980": "immigrant status - immigrants - before 1980",
        "    Employee": "workers - employee - total",
        "    Migrants": "migrants 5 years - movers - migrants - total",
        "    Non-migrants": "migrants 5 years - movers - non-migrants",
        "    Self-employed": "workers - self-employed",
        "  % of owner households spending 30% or more of its income on shelter costs": "dwelling owners (%) - spending over 30% in shelter",
        "  % of owner households with a mortgage": "dwelling owners (%) - with mortgage",
        "  % of tenant households in subsidized housing": "dwelling tenants (%) - in subsidized housing",
        "  % of households living in a dwelling provided by the local government, First Nation or Indian band spending more than 30% on shelter costs": "dwelling government (%) - spending over 30% in shelter",
        "  % of tenant households spending 30% or more of its income on shelter costs": "dwelling tenants (%) - spending over 30% in shelter",
        "  0 to 14 years": "age group (%) - 0-14 years",
        "  0 to 17 years (%)": "LICO-AT (%) - 0-17 years",
        "  1 person": "household size - 1 person",
        "  15 to 64 years": "age group (%) - 15-64 years",
        "  18 to 64 years (%)": "LICO-AT (%) - 18-64 years",
        "  2 persons": "household size - 2 persons",
        "  3 persons": "household size - 3 persons",
        "  4 persons": "household size - 4 persons",
        "  5 or more persons": "household size - 5 or more persons",
        "  65 years and over": "age group (%) - 65+ years",
        "  65 years and over (%)": "LICO-AT (%) - 65+ years",
        "  All classes of workers": "workers - total",
        "  Apartment in a building that has fewer than five storeys": "type of dwelling - apartment in a building that has fewer than five storeys",
        "  Apartment in a building that has five or more storeys": "type of dwelling - apartment in a building that has five or more storeys",
        "  Apartment or flat in a duplex": "type of dwelling - apartment or flat in a duplex",
        "  Apartment or flat in a multi-family building": "type of dwelling - apartment or flat in a multi-family building",
        "  Median after-tax income of household in 2020 ($)": "after-tax income of household ($) - average",
        "  Average after-tax income of household in 2020 ($)": "after-tax income of household ($) - average",
        "  Average total income of household in 2020 ($)": "total income of household ($) - average",
        "  Median total income of household in 2020 ($)": "total income of household ($) - median",
        "  Gini index on adjusted household after-tax income": "Gini index - adjusted household after-tax income",
        "  Gini index on adjusted household market income": "Gini index - adjusted household market income",
        "  Gini index on adjusted household total income": "Gini index - adjusted household total income",
        "  High (secondary) school diploma or equivalency certificate": "education - high school or equivalent",
        "  Immigrants": "immigrant status - immigrants - total",
        "  Median monthly shelter costs for owned dwellings ($)": "dwelling owners ($) - median shelter costs",
        "  Median monthly shelter costs for rented dwellings ($)": "dwelling tenants ($) - median shelter costs",
        "  Median monthly shelter costs for dwellings provided by local government, First Nation or Indian band ($)": "dwelling government - median shelter costs",
        "  Median value of dwellings ($)": "dwelling owners ($) - median value of dwelling",
        "  Movable dwelling": "type of dwelling - moving dwelling",
        "  Movers": "migrant 5 years - movers - total",
        "  No certificate, diploma or degree": "education - no certificate",
        "  Non-immigrants": "immigrant status - non-immigrants",
        "  Non-movers": "migrant 5 years - non-movers",
        "  Non-permanent residents": "immigrant status - non-permanant residents",
        "  Other single-attached house": "type of dwelling - other single-attached house",
        "  P90/P10 ratio on adjusted household after-tax income": "P90/P10 - adjusted household after-tax income",
        "  Postsecondary certificate, diploma or degree": "education - post secondary",
        "  Row house": "type of dwelling - rowhouse",
        "  Semi-detached house": "type of dwelling - semi-detached house",
        "  Single-detached house": "type of dwelling - single-detached house",
        "Average household size": "household size - average",
        "Employment rate": "labor force (%) - employment rate",
        "Median age of the population": "age group - median age",
        "Participation rate": "labor force (%) - participation rate",
        "Population percentage change, 2016 to 2021": "population - change from 2016-2021",
        "Population, 2021": "population - number",
        "Prevalence of low income based on the Low-income cut-offs, after tax (LICO-AT) (%)": "LICO-AT (%) - All",
        "Private dwellings occupied by usual residents": "private dwelling - user occupied",
        "Total - Highest certificate, diploma or degree for the population aged 25 to 64 years in private households - 25% sample data": "education - total",
        "Total - Households living in a dwelling provided by the local government, First Nation or Indian band in non-farm private dwellings - 25% sample data": "dwelling government - total",
        "Total - Immigrant status and period of immigration for the population in private households - 25% sample data": "immigrant status - total",
        "Total - Mobility status 5 years ago - 25% sample data": "migrant 5 years - total",
        "Total - Occupied private dwellings by structural type of dwelling - 100% data": "type of dwelling - total",
        "Total - Owner households in non-farm, non-reserve private dwellings - 25% sample data": "dwelling owners - total",
        "Total - Private households by household size - 100% data": "household size - total",
        "Total - Tenant households in non-farm, non-reserve private dwellings - 25% sample data": "dwelling tenants - total",
        "Total private dwellings": "private dwelling - total",
        "Unemployment rate": "labor force (%) - unemployment rate",
        "GEO_NAME	": "city",
    },
    inplace=True,
)

# Make sure sums are correct
type_of_dwelling = [
    "type of dwelling - apartment in a building that has fewer than five storeys",
    "type of dwelling - apartment in a building that has five or more storeys",
    "type of dwelling - apartment or flat in a duplex",
    "type of dwelling - moving dwelling",
    "type of dwelling - other single-attached house",
    "type of dwelling - rowhouse",
    "type of dwelling - semi-detached house",
    "type of dwelling - single-detached house",
]

df_census["type of dwelling - total"] = df_census[type_of_dwelling].sum(axis=1)

df_census[type_of_dwelling] = (
    df_census[type_of_dwelling].div(df_census["type of dwelling - total"], axis=0) * 100
).apply(lambda x: round(x, 1))


##

household_size = [
    "household size - 1 person",
    "household size - 2 persons",
    "household size - 3 persons",
    "household size - 4 persons",
    "household size - 5 or more persons",
]

df_census["household size - total"] = df_census[household_size].sum(axis=1)

df_census[household_size] = (
    df_census[household_size].div(df_census["household size - total"], axis=0) * 100
).apply(lambda x: round(x, 1))

##

immigrant_status_1 = [
    "immigrant status - immigrants - 1980-1990",
    "immigrant status - immigrants - 1991-2000",
    "immigrant status - immigrants - 2001-2010",
    "immigrant status - immigrants - 2011-2015",
    "immigrant status - immigrants - 2016-202l",
    "immigrant status - immigrants - before 1980",
]

df_census["immigrant status - immigrants - total"] = df_census[immigrant_status_1].sum(
    axis=1
)

immigrant_status_2 = [
    "immigrant status - immigrants - total",
    "immigrant status - non-permanant residents",
    "immigrant status - non-immigrants",
]

df_census["immigrant status - total"] = df_census[immigrant_status_2].sum(axis=1)

immigrant_status_3 = [
    "immigrant status - immigrants - 1980-1990",
    "immigrant status - immigrants - 1991-2000",
    "immigrant status - immigrants - 2001-2010",
    "immigrant status - immigrants - 2011-2015",
    "immigrant status - immigrants - 2016-202l",
    "immigrant status - immigrants - before 1980",
    "immigrant status - non-permanant residents",
    "immigrant status - non-immigrants",
]

df_census[immigrant_status_3] = (
    df_census[immigrant_status_3].div(df_census["immigrant status - total"], axis=0)
    * 100
).apply(lambda x: round(x, 1))

##

migrants_1 = [
    "migrant 5 years - movers - migrants - external",
    "migrant 5 years - movers - migrants - internal",
]

df_census["migrants 5 years - movers - migrants - total"] = df_census[migrants_1].sum(
    axis=1
)


migrants_2 = [
    "migrants 5 years - movers - migrants - total",
    "migrants 5 years - movers - non-migrants",
]

df_census["migrant 5 years - movers - total"] = df_census[migrants_2].sum(axis=1)

migrants_3 = ["migrant 5 years - non-movers", "migrant 5 years - movers - total"]

migrants_4 = [
    "migrant 5 years - movers - migrants - external",
    "migrant 5 years - movers - migrants - internal",
    "migrants 5 years - movers - non-migrants",
    "migrant 5 years - non-movers",
]

df_census["migrant 5 years - total"] = df_census[migrants_3].sum(axis=1)

df_census[migrants_4] = (
    df_census[migrants_4].div(df_census["migrant 5 years - total"], axis=0) * 100
).apply(lambda x: round(x, 1))

##

education = [
    "education - high school or equivalent",
    "education - no certificate",
    "education - post secondary",
]

df_census["education - total"] = df_census[education].sum(axis=1)

df_census[education] = (
    df_census[education].div(df_census["education - total"], axis=0) * 100
).apply(lambda x: round(x, 1))

##

workers_1 = ["workers - employee - temporary", "workers - empoyee - permanent"]

df_census["workers - employee - total"] = df_census[workers_1].sum(axis=1)

workers_2 = ["workers - self-employed", "workers - employee - total"]

df_census["workers - total"] = df_census[workers_2].sum(axis=1)

workers_3 = [
    "workers - employee - temporary",
    "workers - empoyee - permanent",
    "workers - self-employed",
]

df_census[workers_3] = (
    df_census[workers_3].div(df_census["workers - total"], axis=0) * 100
).apply(lambda x: round(x, 1))

##

ownership = [
    "dwelling government - total",
    "dwelling owners - total",
    "dwelling tenants - total",
]

df_census["dwelling ownership - total"] = df_census[ownership].sum(axis=1)
df_census[ownership] = (
    df_census[ownership].div(df_census["dwelling ownership - total"], axis=0) * 100
).apply(lambda x: round(x, 1))

##

df_census["private dwelling - user occupied"] = (
    df_census["private dwelling - user occupied"].div(
        df_census["private dwelling - total"], axis=0
    )
    * 100
).apply(lambda x: round(x, 1))

# drop all totals

totals = [
    "type of dwelling - total",
    "household size - total",
    "immigrant status - immigrants - total",
    "immigrant status - total",
    "migrants 5 years - movers - migrants - total",
    "migrant 5 years - movers - total",
    "migrant 5 years - total",
    "education - total",
    "workers - employee - total",
    "workers - total",
    "type of dwelling - total",
    "dwelling ownership - total",
    "private dwelling - total",
]

df_census = df_census.drop(columns=totals)

df_census.rename(
    columns={
        "age group - median age": "age group (#) - median age",
        "dwelling government - total": "dwelling - government (%)",
        "dwelling owners - total": "dwelling - owners (%)",
        "dwelling tenants - total": "dwelling - tenants (%)",
        "education - high school or equivalent": "education (%) - high school or equivalent",
        "education - no certificate": "education (%) - no certificate",
        "education - post secondary": "education (%) - post secondary",
        "household size - 1 person": "household size (%) - 1 person",
        "household size - 2 persons": "household size (%) - 2 persons",
        "household size - 3 persons": "household size (%) - 3 persons",
        "household size - 4 persons": "household size (%) - 4 persons",
        "household size - 5 or more persons": "household size (%) - 5 or more persons",
        "household size - average": "household size (#) - average",
        "immigrant status - immigrants - 1980-1990": "immigrant status (%) - immigrants - 1980-1990",
        "immigrant status - immigrants - 1991-2000": "immigrant status (%) - immigrants - 1991-2000",
        "immigrant status - immigrants - 2001-2010": "immigrant status (%) - immigrants - 2001-2010",
        "immigrant status - immigrants - 2011-2015": "immigrant status (%) - immigrants - 2011-2015",
        "immigrant status - immigrants - 2016-202l": "immigrant status (%) - immigrants - 2016-202l",
        "immigrant status - immigrants - before 1980": "immigrant status (%) - immigrants - before 1980",
        "immigrant status - non-immigrants": "immigrant status (%) - non-immigrants",
        "immigrant status - non-permanant residents": "immigrant status (%) - non-permanant residents",
        "migrant 5 years - movers - migrants - external": "migrant 5 years (%) - movers - migrants - external",
        "migrant 5 years - movers - migrants - internal": "migrant 5 years (%) - movers - migrants - internal",
        "migrant 5 years - non-movers": "migrant 5 years (%) - non-movers",
        "migrants 5 years - movers - non-migrants": "migrants 5 years (%) - movers - non-migrants",
        "private dwelling - user occupied": "private dwelling (%) - user occupied",
        "population - change from 2016-2021": "population (%) - change from 2016-2021",
        "population - number": "population (#) - number",
        "type of dwelling - apartment in a building that has fewer than five storeys": "type of dwelling (%) - apartment in a building that has fewer than five storeys",
        "type of dwelling - apartment in a building that has five or more storeys": "type of dwelling (%) - apartment in a building that has five or more storeys",
        "type of dwelling - apartment or flat in a duplex": "type of dwelling (%) - apartment or flat in a duplex",
        "type of dwelling - moving dwelling": "type of dwelling (%) - moving dwelling",
        "type of dwelling - other single-attached house": "type of dwelling (%) - other single-attached house",
        "type of dwelling - rowhouse": "type of dwelling (%) - rowhouse",
        "type of dwelling - semi-detached house": "type of dwelling (%) - semi-detached house",
        "type of dwelling - single-detached house": "type of dwelling (%) - single-detached house",
        "workers - employee - temporary": "workers (%) - employee - temporary",
        "workers - empoyee - permanent": "workers (%) - empoyee - permanent",
        "workers - self-employed": "workers (%) - self-employed",
    },
    inplace=True,
)

df_census = df_census.reindex(sorted(df_census.columns), axis=1)

df_census


In [ ]:
# merge datasets

df.drop(df.index, inplace=True)

# merge shelter, census and weather datasets
df = pd.merge(df_alberta, df_census, on="city")
df = pd.merge(df, df_weather, on=["city", "date"])

# merge inflation dataset
df["month"] = df["date"].dt.strftime("%Y-%m")
df_infl["month"] = df_infl["date"].dt.strftime("%Y-%m")
df_empl["month"] = df_empl["date"].dt.strftime("%Y-%m")
df_inflca["month"] = df_inflca["date"].dt.strftime("%Y-%m")

df_infl = df_infl.drop(columns=["date"])
df_empl = df_empl.drop(columns=["date"])
df_inflca = df_inflca.drop(columns=["date"])

df["CMA"] = df["city"]
df.loc[~df["CMA"].isin(["Edmonton", "Calgary"]), "CMA"] = "AB"

df = pd.merge(df, df_infl, on=["CMA", "month"])
df = pd.merge(df, df_empl, on=["province", "month"])
df = pd.merge(df, df_inflca, on=["month"])

df


In [ ]:
df_inflca


In [ ]:
# prompt: list columns in df

print(df_inflca.columns)


In [ ]:
# prompt: export df to excel

# Assuming 'df' is the DataFrame you want to export
# Set index=False to exclude row numbers
df.to_excel(path + "output_alberta_2.xlsx", index=False)


In [ ]:
# Define the local repository path
repo_dir = r"C:\Users\AShresth\Data-Challenge"

# Define the path to the 'Toronto' folder within the repository
path = os.path.join(repo_dir, "Toronto")

# Load the Toronto shelter occupancy data for the years 2021 to 2024
df_toronto_2021 = pd.read_csv(
    os.path.join(path, "daily-shelter-overnight-service-occupancy-capacity-2021.csv")
)
df_toronto_2021["OCCUPANCY_DATE"] = "20" + df_toronto_2021["OCCUPANCY_DATE"].astype(str)
df_toronto_2021["OCCUPANCY_DATE"] = pd.to_datetime(df_toronto_2021["OCCUPANCY_DATE"])

df_toronto_2022 = pd.read_csv(
    os.path.join(path, "daily-shelter-overnight-service-occupancy-capacity-2022.csv")
)
df_toronto_2022["OCCUPANCY_DATE"] = "20" + df_toronto_2022["OCCUPANCY_DATE"].astype(str)
df_toronto_2022["OCCUPANCY_DATE"] = pd.to_datetime(df_toronto_2022["OCCUPANCY_DATE"])

df_toronto_2023 = pd.read_csv(
    os.path.join(path, "daily-shelter-overnight-service-occupancy-capacity-2023.csv")
)
df_toronto_2023["OCCUPANCY_DATE"] = pd.to_datetime(df_toronto_2023["OCCUPANCY_DATE"])

df_toronto_2024 = pd.read_csv(
    os.path.join(path, "Daily shelter overnight occupancy.csv")
)
df_toronto_2024["OCCUPANCY_DATE"] = pd.to_datetime(df_toronto_2024["OCCUPANCY_DATE"])

# Concatenate all the data
df_toronto = pd.concat(
    [df_toronto_2021, df_toronto_2022, df_toronto_2023, df_toronto_2024],
    axis=0,
    ignore_index=True,
)

# Find day of the week
df_toronto["day_week"] = df_toronto["OCCUPANCY_DATE"].dt.day_of_week
df_toronto["day_week"].replace(
    {
        0: "1-mon",
        1: "2-tue",
        2: "3-wed",
        3: "4-thu",
        4: "5-fri",
        5: "6-sat",
        6: "7-sun",
    },
    inplace=True,
)

# Find month
df_toronto["month"] = df_toronto["OCCUPANCY_DATE"].dt.month
df_toronto["month"].replace(
    {
        1: "01-jan",
        2: "02-feb",
        3: "03-mar",
        4: "04-apr",
        5: "05-may",
        6: "06-jun",
        7: "07-jul",
        8: "08-aug",
        9: "09-sep",
        10: "10-oct",
        11: "11-nov",
        12: "12-dec",
    },
    inplace=True,
)

# Find year
df_toronto["year"] = df_toronto["OCCUPANCY_DATE"].dt.year

# Find the last week of the month
df_toronto["Date_Month_End"] = df_toronto["OCCUPANCY_DATE"] + pd.offsets.MonthEnd(n=0)
df_toronto["Date_Month_End"] = df_toronto["Date_Month_End"].dt.day
df_toronto["day_month"] = df_toronto["OCCUPANCY_DATE"].dt.day
df_toronto["last_week"] = np.where(
    (df_toronto["Date_Month_End"] - df_toronto["day_month"]) < 8, 1, 0
)

# Find the first week of the month
df_toronto["first_week"] = np.where(df_toronto["day_month"] < 8, 1, 0)

# Bed capacity
df_toronto.fillna(0, inplace=True)
df_toronto["Capacity"] = (
    df_toronto["CAPACITY_ACTUAL_BED"] + df_toronto["CAPACITY_ACTUAL_ROOM"]
)
df_toronto["Overnight"] = df_toronto["OCCUPIED_BEDS"] + df_toronto["OCCUPIED_ROOMS"]
df_toronto["unoccupied"] = (
    df_toronto["UNOCCUPIED_BEDS"] + df_toronto["UNOCCUPIED_ROOMS"]
)
df_toronto["occupancy_rate"] = (
    df_toronto["Capacity"] - df_toronto["unoccupied"]
) / df_toronto["Capacity"]

# Rename columns
df_toronto["Organization"] = df_toronto["ORGANIZATION_NAME"]
df_toronto["ShelterName"] = df_toronto["LOCATION_NAME"]
df_toronto["ShelterType"] = df_toronto["OVERNIGHT_SERVICE_TYPE"]
df_toronto["city"] = df_toronto["LOCATION_CITY"]
df_toronto["date"] = df_toronto["OCCUPANCY_DATE"].dt.date

# Remove closed shelters
df_toronto.loc[df_toronto["Capacity"] == 0, "Capacity"] = np.nan

# Final dataframe columns
df_toronto = df_toronto[
    [
        "date",
        "city",
        "ShelterType",
        "ShelterName",
        "Organization",
        "Capacity",
        "Overnight",
        "year",
        "month",
        "unoccupied",
        "occupancy_rate",
        "day_week",
        "last_week",
        "first_week",
    ]
]

# Merge column (if needed)
df_toronto["merge"] = 1

# Ensure 'date' is in datetime format
df_toronto["date"] = df_toronto["date"].astype("datetime64[ns]")

# Display the final dataframe
df_toronto


In [ ]:
# Define the local repository path
repo_dir = r"C:\Users\AShresth\Data-Challenge"

# Change the working directory to your local repository
os.chdir(repo_dir)

# Run the git pull command to update the local repository with the latest
# changes
subprocess.run(["git", "pull", "origin", "main"], check=True)

# Define the path to the census data CSV file within the repository
path = os.path.join(repo_dir, "census_CMA.csv")

# Load the census data CSV
df_census = pd.read_csv(path, encoding="ISO-8859-1")
# keep only toronto

df_census = df_census[df_census["GEO_NAME"] == "Toronto"]

list_characteristics = [
    1,
    3,
    4,
    5,
    35,
    36,
    37,
    40,
    41,
    42,
    43,
    44,
    45,
    46,
    47,
    48,
    49,
    50,
    51,
    52,
    53,
    54,
    55,
    57,
    252,
    253,
    360,
    361,
    363,
    364,
    379,
    380,
    381,
    382,
    1482,
    1483,
    1484,
    1486,
    1488,
    1490,
    1491,
    1492,
    1494,
    1496,
    1497,
    1498,
    1499,
    1527,
    1528,
    1529,
    1530,
    1531,
    1532,
    1533,
    1535,
    1536,
    1537,
    1983,
    1984,
    1985,
    1986,
    1987,
    1988,
    1991,
    2014,
    2015,
    2016,
    2017,
    2228,
    2229,
    2230,
    2239,
    2240,
    2241,
    2242,
    2245,
]
df_census = df_census.loc[df_census["CHARACTERISTIC_ID"].isin(list_characteristics)]

# keep relevant columns
df_census = df_census[
    [
        "CENSUS_YEAR",
        "GEO_NAME",
        "CHARACTERISTIC_ID",
        "CHARACTERISTIC_NAME",
        "C1_COUNT_TOTAL",
    ]
]

df_census.rename(columns={"CENSUS_YEAR": "year", "GEO_NAME": "city"}, inplace=True)

# create pivot table
df_census = pd.pivot_table(
    df_census, values="C1_COUNT_TOTAL", index=["city"], columns=["CHARACTERISTIC_NAME"]
)
df_census = df_census.reset_index()

# clean pivot table
df_census.rename(
    columns={
        "      2011 to 2015": "immigrant status - immigrants - 2011-2015",
        "      2016 to 2021": "immigrant status - immigrants - 2016-202l",
        "      External migrants": "migrant 5 years - movers - migrants - external",
        "      Internal migrants": "migrant 5 years - movers - migrants - internal",
        "      Permanent position": "workers - empoyee - permanent",
        "      Temporary position": "workers - employee - temporary",
        "    1980 to 1990": "immigrant status - immigrants - 1980-1990",
        "    1991 to 2000": "immigrant status - immigrants - 1991-2000",
        "    2001 to 2010": "immigrant status - immigrants - 2001-2010",
        "    Before 1980": "immigrant status - immigrants - before 1980",
        "    Employee": "workers - employee - total",
        "    Migrants": "migrants 5 years - movers - migrants - total",
        "    Non-migrants": "migrants 5 years - movers - non-migrants",
        "    Self-employed": "workers - self-employed",
        "  % of owner households spending 30% or more of its income on shelter costs": "dwelling owners (%) - spending over 30% in shelter",
        "  % of owner households with a mortgage": "dwelling owners (%) - with mortgage",
        "  % of tenant households in subsidized housing": "dwelling tenants (%) - in subsidized housing",
        "  % of households living in a dwelling provided by the local government, First Nation or Indian band spending more than 30% on shelter costs": "dwelling government (%) - spending over 30% in shelter",
        "  % of tenant households spending 30% or more of its income on shelter costs": "dwelling tenants (%) - spending over 30% in shelter",
        "  0 to 14 years": "age group (%) - 0-14 years",
        "  0 to 17 years (%)": "LICO-AT (%) - 0-17 years",
        "  1 person": "household size - 1 person",
        "  15 to 64 years": "age group (%) - 15-64 years",
        "  18 to 64 years (%)": "LICO-AT (%) - 18-64 years",
        "  2 persons": "household size - 2 persons",
        "  3 persons": "household size - 3 persons",
        "  4 persons": "household size - 4 persons",
        "  5 or more persons": "household size - 5 or more persons",
        "  65 years and over": "age group (%) - 65+ years",
        "  65 years and over (%)": "LICO-AT (%) - 65+ years",
        "  All classes of workers": "workers - total",
        "  Apartment in a building that has fewer than five storeys": "type of dwelling - apartment in a building that has fewer than five storeys",
        "  Apartment in a building that has five or more storeys": "type of dwelling - apartment in a building that has five or more storeys",
        "  Apartment or flat in a duplex": "type of dwelling - apartment or flat in a duplex",
        "  Apartment or flat in a multi-family building": "type of dwelling - apartment or flat in a multi-family building",
        "  Median after-tax income of household in 2020 ($)": "after-tax income of household ($) - average",
        "  Average after-tax income of household in 2020 ($)": "after-tax income of household ($) - average",
        "  Average total income of household in 2020 ($)": "total income of household ($) - average",
        "  Median total income of household in 2020 ($)": "total income of household ($) - median",
        "  Gini index on adjusted household after-tax income": "Gini index - adjusted household after-tax income",
        "  Gini index on adjusted household market income": "Gini index - adjusted household market income",
        "  Gini index on adjusted household total income": "Gini index - adjusted household total income",
        "  High (secondary) school diploma or equivalency certificate": "education - high school or equivalent",
        "  Immigrants": "immigrant status - immigrants - total",
        "  Median monthly shelter costs for owned dwellings ($)": "dwelling owners ($) - median shelter costs",
        "  Median monthly shelter costs for rented dwellings ($)": "dwelling tenants ($) - median shelter costs",
        "  Median monthly shelter costs for dwellings provided by local government, First Nation or Indian band ($)": "dwelling government - median shelter costs",
        "  Median value of dwellings ($)": "dwelling owners ($) - median value of dwelling",
        "  Movable dwelling": "type of dwelling - moving dwelling",
        "  Movers": "migrant 5 years - movers - total",
        "  No certificate, diploma or degree": "education - no certificate",
        "  Non-immigrants": "immigrant status - non-immigrants",
        "  Non-movers": "migrant 5 years - non-movers",
        "  Non-permanent residents": "immigrant status - non-permanant residents",
        "  Other single-attached house": "type of dwelling - other single-attached house",
        "  P90/P10 ratio on adjusted household after-tax income": "P90/P10 - adjusted household after-tax income",
        "  Postsecondary certificate, diploma or degree": "education - post secondary",
        "  Row house": "type of dwelling - rowhouse",
        "  Semi-detached house": "type of dwelling - semi-detached house",
        "  Single-detached house": "type of dwelling - single-detached house",
        "Average household size": "household size - average",
        "Employment rate": "labor force (%) - employment rate",
        "Median age of the population": "age group - median age",
        "Participation rate": "labor force (%) - participation rate",
        "Population percentage change, 2016 to 2021": "population - change from 2016-2021",
        "Population, 2021": "population - number",
        "Prevalence of low income based on the Low-income cut-offs, after tax (LICO-AT) (%)": "LICO-AT (%) - All",
        "Private dwellings occupied by usual residents": "private dwelling - user occupied",
        "Total - Highest certificate, diploma or degree for the population aged 25 to 64 years in private households - 25% sample data": "education - total",
        "Total - Households living in a dwelling provided by the local government, First Nation or Indian band in non-farm private dwellings - 25% sample data": "dwelling government - total",
        "Total - Immigrant status and period of immigration for the population in private households - 25% sample data": "immigrant status - total",
        "Total - Mobility status 5 years ago - 25% sample data": "migrant 5 years - total",
        "Total - Occupied private dwellings by structural type of dwelling - 100% data": "type of dwelling - total",
        "Total - Owner households in non-farm, non-reserve private dwellings - 25% sample data": "dwelling owners - total",
        "Total - Private households by household size - 100% data": "household size - total",
        "Total - Tenant households in non-farm, non-reserve private dwellings - 25% sample data": "dwelling tenants - total",
        "Total private dwellings": "private dwelling - total",
        "Unemployment rate": "labor force (%) - unemployment rate",
        "GEO_NAME	": "city",
    },
    inplace=True,
)

# Make sure sums are correct
type_of_dwelling = [
    "type of dwelling - apartment in a building that has fewer than five storeys",
    "type of dwelling - apartment in a building that has five or more storeys",
    "type of dwelling - apartment or flat in a duplex",
    "type of dwelling - moving dwelling",
    "type of dwelling - other single-attached house",
    "type of dwelling - rowhouse",
    "type of dwelling - semi-detached house",
    "type of dwelling - single-detached house",
]

df_census["type of dwelling - total"] = df_census[type_of_dwelling].sum(axis=1)

df_census[type_of_dwelling] = (
    df_census[type_of_dwelling].div(df_census["type of dwelling - total"], axis=0) * 100
).apply(lambda x: round(x, 1))


##

household_size = [
    "household size - 1 person",
    "household size - 2 persons",
    "household size - 3 persons",
    "household size - 4 persons",
    "household size - 5 or more persons",
]

df_census["household size - total"] = df_census[household_size].sum(axis=1)

df_census[household_size] = (
    df_census[household_size].div(df_census["household size - total"], axis=0) * 100
).apply(lambda x: round(x, 1))

##

immigrant_status_1 = [
    "immigrant status - immigrants - 1980-1990",
    "immigrant status - immigrants - 1991-2000",
    "immigrant status - immigrants - 2001-2010",
    "immigrant status - immigrants - 2011-2015",
    "immigrant status - immigrants - 2016-202l",
    "immigrant status - immigrants - before 1980",
]

df_census["immigrant status - immigrants - total"] = df_census[immigrant_status_1].sum(
    axis=1
)

immigrant_status_2 = [
    "immigrant status - immigrants - total",
    "immigrant status - non-permanant residents",
    "immigrant status - non-immigrants",
]

df_census["immigrant status - total"] = df_census[immigrant_status_2].sum(axis=1)

immigrant_status_3 = [
    "immigrant status - immigrants - 1980-1990",
    "immigrant status - immigrants - 1991-2000",
    "immigrant status - immigrants - 2001-2010",
    "immigrant status - immigrants - 2011-2015",
    "immigrant status - immigrants - 2016-202l",
    "immigrant status - immigrants - before 1980",
    "immigrant status - non-permanant residents",
    "immigrant status - non-immigrants",
]

df_census[immigrant_status_3] = (
    df_census[immigrant_status_3].div(df_census["immigrant status - total"], axis=0)
    * 100
).apply(lambda x: round(x, 1))

##

migrants_1 = [
    "migrant 5 years - movers - migrants - external",
    "migrant 5 years - movers - migrants - internal",
]

df_census["migrants 5 years - movers - migrants - total"] = df_census[migrants_1].sum(
    axis=1
)


migrants_2 = [
    "migrants 5 years - movers - migrants - total",
    "migrants 5 years - movers - non-migrants",
]

df_census["migrant 5 years - movers - total"] = df_census[migrants_2].sum(axis=1)

migrants_3 = ["migrant 5 years - non-movers", "migrant 5 years - movers - total"]

migrants_4 = [
    "migrant 5 years - movers - migrants - external",
    "migrant 5 years - movers - migrants - internal",
    "migrants 5 years - movers - non-migrants",
    "migrant 5 years - non-movers",
]

df_census["migrant 5 years - total"] = df_census[migrants_3].sum(axis=1)

df_census[migrants_4] = (
    df_census[migrants_4].div(df_census["migrant 5 years - total"], axis=0) * 100
).apply(lambda x: round(x, 1))

##

education = [
    "education - high school or equivalent",
    "education - no certificate",
    "education - post secondary",
]

df_census["education - total"] = df_census[education].sum(axis=1)

df_census[education] = (
    df_census[education].div(df_census["education - total"], axis=0) * 100
).apply(lambda x: round(x, 1))

##

workers_1 = ["workers - employee - temporary", "workers - empoyee - permanent"]

df_census["workers - employee - total"] = df_census[workers_1].sum(axis=1)

workers_2 = ["workers - self-employed", "workers - employee - total"]

df_census["workers - total"] = df_census[workers_2].sum(axis=1)

workers_3 = [
    "workers - employee - temporary",
    "workers - empoyee - permanent",
    "workers - self-employed",
]

df_census[workers_3] = (
    df_census[workers_3].div(df_census["workers - total"], axis=0) * 100
).apply(lambda x: round(x, 1))

##

ownership = [
    "dwelling government - total",
    "dwelling owners - total",
    "dwelling tenants - total",
]

df_census["dwelling ownership - total"] = df_census[ownership].sum(axis=1)
df_census[ownership] = (
    df_census[ownership].div(df_census["dwelling ownership - total"], axis=0) * 100
).apply(lambda x: round(x, 1))

##

df_census["private dwelling - user occupied"] = (
    df_census["private dwelling - user occupied"].div(
        df_census["private dwelling - total"], axis=0
    )
    * 100
).apply(lambda x: round(x, 1))

# drop all totals

totals = [
    "type of dwelling - total",
    "household size - total",
    "immigrant status - immigrants - total",
    "immigrant status - total",
    "migrants 5 years - movers - migrants - total",
    "migrant 5 years - movers - total",
    "migrant 5 years - total",
    "education - total",
    "workers - employee - total",
    "workers - total",
    "type of dwelling - total",
    "dwelling ownership - total",
    "private dwelling - total",
]

df_census = df_census.drop(columns=totals)

df_census.rename(
    columns={
        "age group - median age": "age group (#) - median age",
        "dwelling government - total": "dwelling - government (%)",
        "dwelling owners - total": "dwelling - owners (%)",
        "dwelling tenants - total": "dwelling - tenants (%)",
        "education - high school or equivalent": "education (%) - high school or equivalent",
        "education - no certificate": "education (%) - no certificate",
        "education - post secondary": "education (%) - post secondary",
        "household size - 1 person": "household size (%) - 1 person",
        "household size - 2 persons": "household size (%) - 2 persons",
        "household size - 3 persons": "household size (%) - 3 persons",
        "household size - 4 persons": "household size (%) - 4 persons",
        "household size - 5 or more persons": "household size (%) - 5 or more persons",
        "household size - average": "household size (#) - average",
        "immigrant status - immigrants - 1980-1990": "immigrant status (%) - immigrants - 1980-1990",
        "immigrant status - immigrants - 1991-2000": "immigrant status (%) - immigrants - 1991-2000",
        "immigrant status - immigrants - 2001-2010": "immigrant status (%) - immigrants - 2001-2010",
        "immigrant status - immigrants - 2011-2015": "immigrant status (%) - immigrants - 2011-2015",
        "immigrant status - immigrants - 2016-202l": "immigrant status (%) - immigrants - 2016-202l",
        "immigrant status - immigrants - before 1980": "immigrant status (%) - immigrants - before 1980",
        "immigrant status - non-immigrants": "immigrant status (%) - non-immigrants",
        "immigrant status - non-permanant residents": "immigrant status (%) - non-permanant residents",
        "migrant 5 years - movers - migrants - external": "migrant 5 years (%) - movers - migrants - external",
        "migrant 5 years - movers - migrants - internal": "migrant 5 years (%) - movers - migrants - internal",
        "migrant 5 years - non-movers": "migrant 5 years (%) - non-movers",
        "migrants 5 years - movers - non-migrants": "migrants 5 years (%) - movers - non-migrants",
        "private dwelling - user occupied": "private dwelling (%) - user occupied",
        "population - change from 2016-2021": "population (%) - change from 2016-2021",
        "population - number": "population (#) - number",
        "type of dwelling - apartment in a building that has fewer than five storeys": "type of dwelling (%) - apartment in a building that has fewer than five storeys",
        "type of dwelling - apartment in a building that has five or more storeys": "type of dwelling (%) - apartment in a building that has five or more storeys",
        "type of dwelling - apartment or flat in a duplex": "type of dwelling (%) - apartment or flat in a duplex",
        "type of dwelling - moving dwelling": "type of dwelling (%) - moving dwelling",
        "type of dwelling - other single-attached house": "type of dwelling (%) - other single-attached house",
        "type of dwelling - rowhouse": "type of dwelling (%) - rowhouse",
        "type of dwelling - semi-detached house": "type of dwelling (%) - semi-detached house",
        "type of dwelling - single-detached house": "type of dwelling (%) - single-detached house",
        "workers - employee - temporary": "workers (%) - employee - temporary",
        "workers - empoyee - permanent": "workers (%) - empoyee - permanent",
        "workers - self-employed": "workers (%) - self-employed",
    },
    inplace=True,
)

df_census = df_census.reindex(sorted(df_census.columns), axis=1)

df_census["merge"] = 1

df_census


In [ ]:
# Define the local repository path
repo_dir = r"C:\Users\AShresth\Data-Challenge"

# Change the working directory to your local repository
os.chdir(repo_dir)

# Run the git pull command to update the local repository with the latest
# changes
subprocess.run(["git", "pull", "origin", "main"], check=True)

# Define the path to the weather folder within the repository
weather_path = os.path.join(repo_dir, "weather", "*.csv")

# Merge all weather CSV files inside the weather folder
weather_files = glob.glob(weather_path)

# Load and concatenate all weather files into a single DataFrame
df_weather = pd.concat(
    [pd.read_csv(f, encoding="ISO-8859-1") for f in weather_files], ignore_index=True
)

# Remove entries earlier than 2014
df_weather = df_weather[df_weather["LOCAL_YEAR"] >= 2014]

# Rename station names to city names
df_weather["STATION_NAME"].replace({"TORONTO CITY CENTRE": "Toronto"}, inplace=True)

# Remove time from date
df_weather["LOCAL_DATE"] = pd.to_datetime(df_weather["LOCAL_DATE"]).dt.date

# Average out missing temperature from temperatures from day before and after
df_weather = df_weather.sort_values(by=["STATION_NAME", "LOCAL_DATE"])
df_weather["MIN_TEMPERATURE"] = df_weather["MIN_TEMPERATURE"].interpolate()
df_weather["MAX_TEMPERATURE"] = df_weather["MAX_TEMPERATURE"].interpolate()
df_weather["MEAN_TEMPERATURE"] = df_weather["MEAN_TEMPERATURE"].interpolate()

# Replace precipitation by 0 if missing
df_weather.update(df_weather[["TOTAL_PRECIPITATION"]].fillna(0))

# Keep relevant columns
df_weather = df_weather[
    [
        "STATION_NAME",
        "MEAN_TEMPERATURE",
        "TOTAL_PRECIPITATION",
        "LOCAL_DATE",
        "MIN_TEMPERATURE",
        "MAX_TEMPERATURE",
        "PROVINCE_CODE",
        "x",
        "y",
    ]
]

# Rename columns for consistency
df_weather.rename(
    columns={
        "STATION_NAME": "city",
        "MEAN_TEMPERATURE": "mean_temperature",
        "TOTAL_PRECIPITATION": "total_precipitation",
        "LOCAL_DATE": "date",
        "MIN_TEMPERATURE": "min_temperature",
        "MAX_TEMPERATURE": "max_temperature",
        "PROVINCE_CODE": "province",
    },
    inplace=True,
)

# Convert date column to datetime format
df_weather["date"] = df_weather["date"].astype("datetime64[ns]")

# Add a merge column
df_weather["merge"] = 1

# Print the final DataFrame
print(df_weather)


In [ ]:
# merge datasets
path = path = r"C:\Users\AShresth\Data-Challenge"

# Ensure 'date' is in datetime format
df_toronto["date"] = pd.to_datetime(df_toronto["date"])
df_weather["date"] = pd.to_datetime(df_weather["date"])

# Merge the dataframes
df = pd.merge(df_toronto, df_weather, on=["merge", "date"], how="outer")

# merge inflation dataset
df["month"] = df["date"].dt.strftime("%Y-%m")
df_infl["month"] = df_infl["date"].dt.strftime("%Y-%m")
df_empl["month"] = df_empl["date"].dt.strftime("%Y-%m")
df_inflca["month"] = df_inflca["date"].dt.strftime("%Y-%m")

df_infl = df_infl.drop(columns=["date"])
df_empl = df_empl.drop(columns=["date"])
df_inflca = df_inflca.drop(columns=["date"])

df["CMA"] = "Toronto"

df = pd.merge(df, df_infl, on=["CMA", "month"])
df = pd.merge(df, df_empl, on=["province", "month"])
df = pd.merge(df, df_inflca, on=["month"])

df


In [ ]:
# Export

# Set index=False to exclude row numbers
df.to_excel(path + "output_toronto.xlsx", index=False)

df


In [ ]:
# analyze Alberta data


# Define the local repository path
repo_dir = r"C:\Users\AShresth\Data-Challenge"

# Change the working directory to your local repository
os.chdir(repo_dir)

# Run the git pull command to update the local repository with the latest
# changes
subprocess.run(["git", "pull", "origin", "main"], check=True)

# Define the path to the Excel file within the repository
path = os.path.join(repo_dir, "output_alberta_2.xlsx")

# Load the Excel data into a DataFrame
df_orig = pd.read_excel(path)  # Files with station information

df_orig["holidays"] = (
    pd.Series(df_orig.date)
    .apply(lambda x: holidays.CountryHoliday("CA", subdiv="AB").get(x))
    .values
)
df_orig["holidays"] = df_orig["holidays"].notna().astype(int)


df_orig


In [ ]:
# prompt: if holidays is None then 0 or else 1 change to bool

df_orig["holidays"] = df_orig["holidays"].notna().astype(int)


In [ ]:
# prompt: list average Capacity by ShelterName if year is over 2022 and sort

if "date" in df_orig.columns:
    df_orig["year"] = pd.to_datetime(df_orig["date"]).dt.year
    df_filtered = df_orig[df_orig["year"] > 2022]
    average_capacity = (
        df_filtered.groupby("ShelterName")["Capacity"].mean().sort_values().head(50)
    )
    print(average_capacity)
else:
    print("No 'date' column found in the DataFrame.")


In [ ]:
"""
Alberta Analysis One Shelter
"""


df = df_orig
df["month_year"] = df["date"].dt.strftime("%m")

df["pandemic"] = "Pre-pandemic"
df.loc[
    (df["date"] >= "2020-03-11") & (df["date"] <= "2022-10-01"), "pandemic"
] = "Pandemic"
df.loc[(df["date"] > "2022-10-01"), "pandemic"] = "Post-pandemic"
df["deep_cold"] = 0
df.loc[(df["min_temperature"] < -30), "deep_cold"] = 1
df["strong_heat"] = 0
df.loc[(df["max_temperature"] > 38), "strong_heat"] = 1

df = df.loc[df["ShelterName"] == "Alpha House"]

# Line Plot
fig, ax = plt.subplots(figsize=(10, 8))
sns.lineplot(data=df, x="date", y="occupancy_rate")
ax.set_title("Occupancy rate")
plt.show()

# Heat map
df_heatmap = df
df_heatmap = df_heatmap[df_heatmap["pandemic"] == "Post-pandemic"]
df_heatmap = df[
    [
        "occupancy_rate",
        "last_week",
        "first_week",
        "mean_temperature",
        "total_precipitation",
        "min_temperature",
        "max_temperature",
        "inflation_all",
        "inflation_shelter",
        "unemployment rate",
        "adjusted national inflation rate",
        "deep_cold",
        "strong_heat",
        "holidays",
    ]
]
plt.figure(figsize=(10, 8))
ax.set_title("Correlation matrix post-pandemic")
sns.heatmap(df_heatmap.corr(), annot=True, cmap="YlGnBu")


# Box plots

fig, ax = plt.subplots(figsize=(15, 8))
sns.boxplot(data=df, x="day_week", y="occupancy_rate", hue="pandemic")
ax.set_title("Occupancy rate on each day of the week")
plt.show()

fig, ax = plt.subplots(figsize=(15, 8))
sns.boxplot(data=df, x="month_year", y="occupancy_rate", hue="pandemic")
ax.set_title("Occupancy rate on each month of the year")
plt.show()

# Split the data
df = df[df["date"] >= "2022-10-01"]

df = df[df["occupancy_rate"].notna()]
df = df.set_index("date")

# Changed date from 2022-06-20 to 2023-06-20
train = df.loc[df.index < "2023-12-20"]
test = df.loc[df.index >= "2023-12-20"]

# Create model
FEATURES = [
    "last_week",
    "first_week",
    "mean_temperature",
    "total_precipitation",
    "min_temperature",
    "max_temperature",
    "unemployment rate",
    "holidays",
]
TARGET = "occupancy_rate"

X_train = train[FEATURES]
y_train = train[TARGET]

X_test = test[FEATURES]
y_test = test[TARGET]

# Perform the XGB Regression
reg = xgb.XGBRegressor(
    base_score=0.5,
    booster="gbtree",
    n_estimators=1000,
    early_stopping_rounds=50,
    objective="reg:linear",
    max_depth=3,
    learning_rate=0.01,
)

reg.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], verbose=100)

fi = pd.DataFrame(
    data=reg.feature_importances_, index=reg.feature_names_in_, columns=["importance"]
)

fi.sort_values("importance").plot(kind="barh", title="Feature Importance")

plt.show()

# Forecast on test
test["prediction"] = reg.predict(X_test)
df = df.merge(test[["prediction"]], how="left", left_index=True, right_index=True)
ax = df[["occupancy_rate"]].plot(figsize=(15, 5))
df["prediction"].plot(ax=ax)
plt.legend(["Real Data", "Predictions"])
ax.set_title("Real Data and Prediction for Alpha House, Calgary")
plt.xlabel("Date")
plt.ylabel("Occupancy Rate (%)")
plt.show()

# Filter the data for dates after January 1, 2024
df_filtered = df[df.index >= "2024-01-01"]

# Create the plot
ax = df_filtered[["occupancy_rate"]].plot(figsize=(8, 5))
df_filtered["prediction"].plot(ax=ax)

# Customize the plot
plt.legend(["Real Data", "Predictions"])
ax.set_title("Real Data and Prediction for Alpha House, Calgary")
ax.xaxis.set_major_locator(mdates.MonthLocator())
plt.xlabel("Date")
plt.ylabel("Occupancy Rate")
plt.show()

reg.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], verbose=100)

fold = 0
preds = []
scores = []

y_pred = reg.predict(X_test)
preds.append(y_pred)
score = np.sqrt(mean_squared_error(y_test, y_pred))
scores.append(score)

print(f"Score across folds {np.mean(scores):0.4f}")
print(f"Fold scores:{scores}")


In [ ]:
# prompt: plot occupancy rate and prediction on the same graph only after
# january 1, 2024


# Filter the data for dates after January 1, 2024
df_filtered = df[df.index >= "2024-01-01"]

# Create the plot
ax = df_filtered[["occupancy_rate"]].plot(figsize=(15, 5))
df_filtered["prediction"].plot(ax=ax)

# Customize the plot
plt.legend(["Real Data", "Predictions"])
ax.set_title("Real Data and Prediction for Alpha House, Calgary")
plt.xlabel("Date")
plt.ylabel("Occupancy Rate (%)")
plt.show()


In [ ]:
"""
Alberta Analysis One Shelter
"""


df = df_orig
df["month_year"] = df["date"].dt.strftime("%m")

df["pandemic"] = "Pre-pandemic"
df.loc[
    (df["date"] >= "2020-03-11") & (df["date"] <= "2022-10-01"), "pandemic"
] = "Pandemic"
df.loc[(df["date"] > "2022-10-01"), "pandemic"] = "Post-pandemic"
df["deep_cold"] = 0
df.loc[(df["min_temperature"] < -30), "deep_cold"] = 1
df["strong_heat"] = 0
df.loc[(df["max_temperature"] > 38), "strong_heat"] = 1

df = df[df["date"] >= "2023-03-19"]
df = df[df["date"] <= "2023-03-25"]

df = df[df["occupancy_rate"].notna()]
df = df.set_index("date")

# df = df.loc[df['ShelterName'] == 'YW Calgary Emergency']

# Line Plot
fig, ax = plt.subplots(figsize=(10, 8))
sns.lineplot(data=df, x="date", y="occupancy_rate")
ax.set_title("Occupancy rate")
plt.show()

df["avg_occupancy"] = df.groupby(["ShelterName"])["Capacity"].transform("mean")


In [ ]:
if "date" in df_orig.columns:
    df_orig["year"] = pd.to_datetime(df_orig["date"]).dt.year
    df_filtered = df_orig[df_orig["year"] > 2022]
    average_capacity = (
        df_filtered.groupby("ShelterName")["Capacity"].mean().sort_values().head(50)
    )
    print(average_capacity)
else:
    print("No 'date' column found in the DataFrame.")


In [ ]:
"""
Alberta Analysis Every Shelter
"""


df = df_orig
df["month_year"] = df["date"].dt.strftime("%m")

df["pandemic"] = "Pre-pandemic"
df.loc[
    (df["date"] >= "2020-03-11") & (df["date"] <= "2022-10-01"), "pandemic"
] = "Pandemic"
df.loc[(df["date"] > "2022-10-01"), "pandemic"] = "Post-pandemic"
df["deep_cold"] = 0
df.loc[(df["min_temperature"] < -30), "deep_cold"] = 1
df["strong_heat"] = 0
df.loc[(df["max_temperature"] > 38), "strong_heat"] = 1

# keep only shelters represented during every week
df["ShelterName_count"] = df.groupby("ShelterName")["ShelterName"].transform("count")
df = df[df["ShelterName_count"] > 3300]

# Line Plot
fig, ax = plt.subplots(figsize=(10, 8))
sns.lineplot(data=df, x="date", y="occupancy_rate")
ax.set_title("Occupancy rate")
plt.show()

# Heat map
df_heatmap = df
df_heatmap = df_heatmap[df_heatmap["pandemic"] == "Post-pandemic"]
df_heatmap = df[
    [
        "occupancy_rate",
        "last_week",
        "first_week",
        "mean_temperature",
        "total_precipitation",
        "min_temperature",
        "max_temperature",
        "inflation_all",
        "inflation_shelter",
        "unemployment rate",
        "adjusted national inflation rate",
        "deep_cold",
        "strong_heat",
    ]
]
plt.figure(figsize=(10, 8))
ax.set_title("Correlation matrix post-pandemic")
sns.heatmap(df_heatmap.corr(), annot=True, cmap="YlGnBu")

# Box plots
fig, ax = plt.subplots(figsize=(15, 8))
sns.boxplot(data=df, x="day_week", y="occupancy_rate", hue="pandemic")
ax.set_title("Occupancy rate on each day of the week")
plt.show()

fig, ax = plt.subplots(figsize=(15, 8))
sns.boxplot(data=df, x="month_year", y="occupancy_rate", hue="pandemic")
ax.set_title("Occupancy rate on each month of the year")
plt.show()

# Split the data
df = df[df["date"] >= "2022-10-01"]

df = df[df["occupancy_rate"].notna()]
df = df.set_index("date")

# Changed date from 2022-06-20 to 2023-06-20
train = df.loc[df.index < "2023-06-20"]
test = df.loc[df.index >= "2023-06-20"]

# Create model
FEATURES = [
    "last_week",
    "first_week",
    "mean_temperature",
    "total_precipitation",
    "min_temperature",
    "max_temperature",
    "unemployment rate",
    "deep_cold",
    "strong_heat",
]
TARGET = "occupancy_rate"

X_train = train[FEATURES]
y_train = train[TARGET]

X_test = test[FEATURES]
y_test = test[TARGET]

# Perform the XGB Regression
reg = xgb.XGBRegressor(
    base_score=0.5,
    booster="gbtree",
    n_estimators=1000,
    early_stopping_rounds=50,
    objective="reg:linear",
    max_depth=3,
    learning_rate=0.01,
)

reg.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], verbose=100)

fi = pd.DataFrame(
    data=reg.feature_importances_, index=reg.feature_names_in_, columns=["importance"]
)

fi.sort_values("importance").plot(kind="barh", title="Feature Importance")

plt.show()

# Forecast on test
test["prediction"] = reg.predict(X_test)
df = df.merge(test[["prediction"]], how="left", left_index=True, right_index=True)
ax = df[["occupancy_rate"]].plot(figsize=(15, 5))
df["prediction"].plot(ax=ax, style=".")
plt.legend(["Truth Data", "Predictions"])
ax.set_title("Raw Dat and Prediction")
plt.show()


In [ ]:
# analyze Toronto data


# Define the local repository path
repo_dir = r"C:\Users\AShresth\Data-Challenge"

# Change the working directory to your local repository
os.chdir(repo_dir)

# Run the git pull command to update the local repository with the latest
# changes
subprocess.run(["git", "pull", "origin", "main"], check=True)

# Define the path to the Toronto Excel file within the repository
path = os.path.join(repo_dir, "output_toronto.xlsx")

# Load the Toronto data into a DataFrame
df_orig = pd.read_excel(path)  # Files with station information

# Display the DataFrame
df_orig


In [ ]:
"""
Toronto Analysis One Shelter
"""


df = df_orig
df["month_year"] = df["date"].dt.strftime("%m")

df["pandemic"] = "Pre-pandemic"
df.loc[
    (df["date"] >= "2020-03-11") & (df["date"] <= "2022-10-01"), "pandemic"
] = "Pandemic"
df.loc[(df["date"] > "2022-10-01"), "pandemic"] = "Post-pandemic"
df["deep_cold"] = 0
df.loc[(df["min_temperature"] < -30), "deep_cold"] = 1
df["strong_heat"] = 0
df.loc[(df["max_temperature"] > 38), "strong_heat"] = 1

df = df.loc[df["ShelterName"] == "Vanauley Street YMCA Emergency Shelter"]

# Line Plot
fig, ax = plt.subplots(figsize=(10, 8))
sns.lineplot(data=df, x="date", y="occupancy_rate")
ax.set_title("Occupancy rate")
plt.show()

# Heat map
df_heatmap = df
df_heatmap = df_heatmap[df_heatmap["pandemic"] == "Post-pandemic"]
df_heatmap = df[
    [
        "occupancy_rate",
        "last_week",
        "first_week",
        "mean_temperature",
        "total_precipitation",
        "min_temperature",
        "max_temperature",
        "inflation_all",
        "inflation_shelter",
        "unemployment rate",
        "adjusted national inflation rate",
        "deep_cold",
        "strong_heat",
    ]
]
plt.figure(figsize=(10, 8))
ax.set_title("Correlation matrix post-pandemic")
sns.heatmap(df_heatmap.corr(), annot=True, cmap="YlGnBu")

# Box plots

fig, ax = plt.subplots(figsize=(15, 8))
sns.boxplot(data=df, x="day_week", y="occupancy_rate", hue="pandemic")
ax.set_title("Occupancy rate on each day of the week")
plt.show()

fig, ax = plt.subplots(figsize=(15, 8))
sns.boxplot(data=df, x="month_year", y="occupancy_rate", hue="pandemic")
ax.set_title("Occupancy rate on each month of the year")
plt.show()

# Split the data
df = df[df["date"] >= "2022-10-01"]

df = df[df["occupancy_rate"].notna()]
df = df.set_index("date")

# Changed date from 2022-06-20 to 2023-06-20
train = df.loc[df.index < "2023-06-20"]
test = df.loc[df.index >= "2023-06-20"]

# Create model
FEATURES = [
    "last_week",
    "first_week",
    "mean_temperature",
    "total_precipitation",
    "min_temperature",
    "max_temperature",
    "unemployment rate",
    "deep_cold",
    "strong_heat",
]
TARGET = "occupancy_rate"

X_train = train[FEATURES]
y_train = train[TARGET]

X_test = test[FEATURES]
y_test = test[TARGET]

# Perform the XGB Regression
reg = xgb.XGBRegressor(
    base_score=0.5,
    booster="gbtree",
    n_estimators=1000,
    early_stopping_rounds=50,
    objective="reg:linear",
    max_depth=3,
    learning_rate=0.01,
)

reg.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], verbose=100)

fi = pd.DataFrame(
    data=reg.feature_importances_, index=reg.feature_names_in_, columns=["importance"]
)

fi.sort_values("importance").plot(kind="barh", title="Feature Importance")

plt.show()

# Forecast on test
test["prediction"] = reg.predict(X_test)
df = df.merge(test[["prediction"]], how="left", left_index=True, right_index=True)
ax = df[["occupancy_rate"]].plot(figsize=(15, 5))
df["prediction"].plot(ax=ax, style=".")
plt.legend(["Truth Data", "Predictions"])
ax.set_title("Raw Dat and Prediction")
plt.show()


In [ ]:
"""
Toronto Analysis Every Shelter
"""


df = df_orig
df["month_year"] = df["date"].dt.strftime("%m")

df["pandemic"] = "Pre-pandemic"
df.loc[
    (df["date"] >= "2020-03-11") & (df["date"] <= "2022-10-01"), "pandemic"
] = "Pandemic"
df.loc[(df["date"] > "2022-10-01"), "pandemic"] = "Post-pandemic"
df["deep_cold"] = 0
df.loc[(df["min_temperature"] < -30), "deep_cold"] = 1
df["strong_heat"] = 0
df.loc[(df["max_temperature"] > 38), "strong_heat"] = 1

# keep only shelters represented during every week
df["ShelterName_count"] = df.groupby("ShelterName")["ShelterName"].transform("count")

df = df[df["ShelterName_count"] == 1308]

# Line Plot
fig, ax = plt.subplots(figsize=(10, 8))
sns.lineplot(data=df, x="date", y="occupancy_rate")
ax.set_title("Occupancy rate")
plt.show()

# Heat map
df_heatmap = df
df_heatmap = df_heatmap[df_heatmap["pandemic"] == "Post-pandemic"]
df_heatmap = df[
    [
        "occupancy_rate",
        "last_week",
        "first_week",
        "mean_temperature",
        "total_precipitation",
        "min_temperature",
        "max_temperature",
        "inflation_all",
        "inflation_shelter",
        "unemployment rate",
        "adjusted national inflation rate",
        "deep_cold",
        "strong_heat",
    ]
]
plt.figure(figsize=(10, 8))
ax.set_title("Correlation matrix post-pandemic")
sns.heatmap(df_heatmap.corr(), annot=True, cmap="YlGnBu")

# Box plots
fig, ax = plt.subplots(figsize=(15, 8))
sns.boxplot(data=df, x="day_week", y="occupancy_rate", hue="pandemic")
ax.set_title("Occupancy rate on each day of the week")
plt.show()

fig, ax = plt.subplots(figsize=(15, 8))
sns.boxplot(data=df, x="month_year", y="occupancy_rate", hue="pandemic")
ax.set_title("Occupancy rate on each month of the year")
plt.show()

# Split the data
df = df[df["date"] >= "2022-10-01"]

df = df[df["occupancy_rate"].notna()]
df = df.set_index("date")

# Changed date from 2022-06-20 to 2023-06-20
train = df.loc[df.index < "2023-06-20"]
test = df.loc[df.index >= "2023-06-20"]

# Create model
FEATURES = [
    "last_week",
    "first_week",
    "mean_temperature",
    "total_precipitation",
    "min_temperature",
    "max_temperature",
    "unemployment rate",
    "deep_cold",
    "strong_heat",
]
TARGET = "occupancy_rate"

X_train = train[FEATURES]
y_train = train[TARGET]

X_test = test[FEATURES]
y_test = test[TARGET]

# Perform the XGB Regression
reg = xgb.XGBRegressor(
    base_score=0.5,
    booster="gbtree",
    n_estimators=1000,
    early_stopping_rounds=50,
    objective="reg:linear",
    max_depth=3,
    learning_rate=0.01,
)

reg.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], verbose=100)

fi = pd.DataFrame(
    data=reg.feature_importances_, index=reg.feature_names_in_, columns=["importance"]
)

fi.sort_values("importance").plot(kind="barh", title="Feature Importance")

plt.show()

# Forecast on test
test["prediction"] = reg.predict(X_test)
df = df.merge(test[["prediction"]], how="left", left_index=True, right_index=True)
ax = df[["occupancy_rate"]].plot(figsize=(15, 5))
df["prediction"].plot(ax=ax, style=".")
plt.legend(["Truth Data", "Predictions"])
ax.set_title("Raw Dat and Prediction")
plt.show()


In [ ]:
# prompt: count instances of sheltername_count

df_orig["ShelterName_count"].value_counts()
df


In [ ]:
# prompt: find unique values of sheltername if sheltername_count ==1308

df_orig[df_orig["ShelterName_count"] == 1308]["ShelterName"].unique()


#
